Neural Network 102 - Initialization Neural Network

เปรียบเทียบการ Initialize Weight แบบต่าง ๆ ว่าจะมีผลต่อ Gradient อย่างไร แบบไหนช่วยลดปัญหา Vanishing Gradient และ Exploding Gradient

# 0. Magic

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# 1. Import

In [2]:
from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle, gzip, math, torch
from torch import tensor

# 2. Download Dataset

เคสนี้เราจะใช้ MNIST Dataset เหมือนเดิม

In [3]:
MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'

In [4]:
def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train, y_train, x_valid, y_valid))

ดาวน์โหลดชุดข้อมูลมาใส่ x, y ทั้ง Training Set และ Validation Set

In [5]:
x_train, y_train, x_valid, y_valid = get_data()

# 3. เตรียมข้อมูล

หา mean และ standard deviation ของข้อมูล Training Set

In [6]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.1304), tensor(0.3073))

เนื่องจากทุกอย่างมาจากข้อมูลคนละแหล่ง เราไม่ทราบว่าอันไหนสำคัญกว่าอันไหน เราจึง Normalize ข้อมูลทุกอย่างให้อยู่ใน Scale เดียวกันให้หมด โดย mean = 0, std = 1 เท่ากันหมด แล้วให้โมเดลเรียนรู้เอง จากข้อมูลตัวอย่างว่าอันไหน สำคัญเท่าไร สัมพันธ์กันอย่างไร

Normalize คือการทำให้ข้อมูล มี mean = 0 และ standard deviation (std) = 1 ด้วยการ ลบด้วย mean หารด้วย std

In [7]:
# x = data, m = mean, s = standard deviation
def normalize(x, m, s): 
    return (x-m)/s

In [8]:
x_train = normalize(x_train, train_mean, train_std)

Normalize ข้อมูล Validation Set ด้วย mean, std ของ Training Set เพื่อปรับให้เป็น Scale เดียวกัน

In [9]:
x_valid = normalize(x_valid, train_mean, train_std)

ลองดูค่า mean, std หลังจาก Normalize เรียบร้อยแล้ว

In [10]:
train_mean_after, train_std_after = x_train.mean(), x_train.std()
train_mean_after, train_std_after

(tensor(-7.6999e-06), tensor(1.))

ได้ mean เป็นเลขจุดทศนิยมน้อยมาก ๆ ใกล้เคียง 0 และ std ใกล้เคียง 1

n = จำนวน Record, m จำนวน Feature, c = จำนวน class

In [11]:
n, m = x_train.shape
c = y_train.max()+1
n, m, c

(50000, 784, tensor(10))

แปลงจาก Long เป็น Float ก่อนส่งให้ Loss Function เนื่องจากเราใช้ Mse แทน Cross Entropy

In [12]:
y_train, y_valid = y_train.float(), y_valid.float()

# 4. Model

เราจะสร้าง 4 Hidden Layers Neural Networks ที่มี 100 Neuron ในแต่ละ Hidden Layer 

In [13]:
# nh = number of hidden
nh = 100

In [14]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    def forward(self, inp):
        raise Exception('Not implemented')
    def backward(self):
        self.bwd(self.out, *self.args)        

In [15]:
class Relu(Module):
    def forward(self, inp):
        return inp.clamp_min(0.) - 0.5
    def bwd(self, out, inp):
        inp.g = (inp>0).float()*out.g

In [16]:
class Sigmoid(Module):
    def forward(self, inp):
        return 1/(1 + torch.exp(-inp)) 
    def bwd(self, out, inp):
        inp.g = (self.forward(inp) * (1 - self.forward(inp))) * out.g

In [17]:
class Lin(Module):
    def __init__(self, w, b):
        self.w, self.b = w, b
    def forward(self, inp):
        return inp @ self.w + self.b
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.squeeze(-1).t() @ out.g
        self.b.g = out.g.sum(0)

In [18]:
class Mse(Module):
    def forward(self, inp, targ):
        return (inp.squeeze(-1) - targ).pow(2).mean()
    def bwd(self, out, inp, targ):
        inp.g = 2. * (inp.squeeze(-1) - targ).unsqueeze(-1) / inp.shape[0]        

สร้างโมเดล ที่มี 4 Hidden Layer และ Activation หลายแบบให้เลือก ใน Comment

In [19]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2), Relu(), Lin(w3, b3), Relu(), Lin(w4, b4)]
#         self.layers = [Lin(w1, b1), Sigmoid(), Lin(w2, b2), Sigmoid(), Lin(w3, b3), Sigmoid(), Lin(w4, b4)]
#         self.layers = [Lin(w1, b1), Lin(w2, b2), Lin(w3, b3), Lin(w4, b4)]
        self.loss = Mse()
    def __call__(self, x, targ):
#         set_trace()
        for l in self.layers:
            x = l(x)
        return self.loss(x, targ), x
    def backward(self):
        self.loss.backward()        
        for l in reversed(self.layers):
            l.backward()

# 5. Initialize

เราจะลอง Initinalize กำหนดค่าเริ่มต้น Weight และ Bias แบบต่าง ๆ แล้วเราจะใช้ข้อมูลเดียวกัน รันผ่านโมเดล แล้ว Backpropagate เพื่อเปรียบเทียบ ว่า Gradient เป็นอย่างไร  

## 5.1 Normal Distribution (Gaussian)

กำหนดค่าเริ่มต้นของ Weight และ Bias ด้วยค่า Random แบบ Normal Distribution (Gaussian)

In [20]:
w1 = torch.randn(m, nh)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, nh)
b2 = torch.zeros(nh)
w3 = torch.randn(nh, nh)
b3 = torch.zeros(nh)
w4 = torch.randn(nh, 1)
b4 = torch.zeros(1)

ลองดูค่า mean, std ของ Weight

In [21]:
w1.mean(), w1.std()

(tensor(-0.0006), tensor(1.0012))

In [22]:
w2.mean(), w2.std()

(tensor(0.0190), tensor(1.0068))

In [23]:
w3.mean(), w3.std()

(tensor(0.0011), tensor(0.9909))

In [24]:
w4.mean(), w4.std()

(tensor(-0.0879), tensor(1.0427))

Weight ดูดีมาก ใกล้เคียง mean = 0, std = 1

### ลองสมมติว่าเป็น Layer แรก

In [25]:
a1 = x_train @ w1
a1.mean(), a1.std()

(tensor(1.1000), tensor(28.5732))

ลองสมมติว่าเป็น Layer แรก + ReLU Activation

In [26]:
z1 = (x_train @ w1).clamp(0.)-0.5
z1.mean(), z1.std()

(tensor(11.3143), tensor(16.9981))

เราได้ Activation ของ Layer แรก ที่ไม่ใกล้เคียง mean = 0, std = 1 เลย อาจจะมีปัญหาตอนเทรนได้

### สร้างโมเดล รัน forward และ backward 

In [27]:
w1.g,b1.g,w2.g,b2.g,w3.g,b3.g,w4.g,b4.g = [None]*8
model = Model()
loss, yhat = model(x_train, y_train)
model.backward()

ตรวจเช็ค Mean และ Standard Deviation ของ Gradient ของ Weight ของแต่ละ Layer

In [28]:
w4.g.mean(), w4.g.std()

(tensor(6752871.5000), tensor(7748565.5000))

In [29]:
w3.g.mean(), w3.g.std()

(tensor(50369.2070), tensor(899103.5000))

In [30]:
w2.g.mean(), w2.g.std()

(tensor(43154.5938), tensor(557012.8125))

In [31]:
w1.g.mean(), w1.g.std()

(tensor(515.9893), tensor(157482.1406))

Gradient ที่เราต้องการคือประมาณ mean = 0, std = 1 แต่นี่ไม่ได้ใกล้เคียงเลย ถ้าเป็นแบบนี้จะทำให้เกิดปัญหา Exploding Gradient

## 5.2 Kaiming Initialization

กำหนดค่าเริ่มต้นของ Weight และ Bias ด้วย Kaiming Init เวอร์ชันง่าย * math.sqrt(1./m)

In [32]:
w1 = torch.randn(m, nh) * math.sqrt(1./m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, nh)* math.sqrt(1./nh)
b2 = torch.zeros(nh)
w3 = torch.randn(nh, nh) * math.sqrt(1./nh)
b3 = torch.zeros(nh)
w4 = torch.randn(nh, 1) * math.sqrt(1./nh)
b4 = torch.zeros(1)

ลองดูค่า mean, std ของ Weight

In [33]:
w1.mean(), w1.std()

(tensor(-1.0969e-05), tensor(0.0358))

In [34]:
w2.mean(), w2.std()

(tensor(-0.0002), tensor(0.0988))

In [35]:
w3.mean(), w3.std()

(tensor(0.0004), tensor(0.0995))

In [36]:
w4.mean(), w4.std()

(tensor(-0.0047), tensor(0.1064))

Weight จะ mean, std เป็นอย่างไรเรายังไม่สนใจ

### ลองสมมติว่าเป็น Layer แรก

In [37]:
a1 = x_train @ w1
a1.mean(), a1.std()

(tensor(0.0260), tensor(0.9986))

ลองสมมติว่าเป็น Layer แรก + ReLU Activation

In [38]:
z1 = (x_train @ w1).clamp(0.)-0.5
z1.mean(), z1.std()

(tensor(-0.0915), tensor(0.5835))

เราได้ Activation ของ Layer แรก ที่ใกล้เคียง mean = 0, std = 1 มากขึ้น

### สร้างโมเดล

In [39]:
w1.g,b1.g,w2.g,b2.g,w3.g,b3.g,w4.g,b4.g = [None]*8
model = Model()
loss, yhat = model(x_train, y_train)
model.backward()

ตรวจเช็ค Mean และ Standard Deviation ของ Gradient ของ Weight ของแต่ละ Layer

In [40]:
w4.g.mean(), w4.g.std()

(tensor(2.6649), tensor(1.3136))

In [41]:
w3.g.mean(), w3.g.std()

(tensor(0.0009), tensor(0.1455))

In [42]:
w2.g.mean(), w2.g.std()

(tensor(-0.0045), tensor(0.0782))

In [43]:
w1.g.mean(), w1.g.std()

(tensor(0.0001), tensor(0.0831))

เราได้ Gradient ที่ใกล้เคียง mean = 0, std = 1 มากขึ้น

## 5.3 Kaiming Initialization 2

กำหนดค่าเริ่มต้นของ Weight และ Bias ด้วย Kaiming Init เวอร์ชันปรับปรุง เปลี่ยนจาก 1./m เป็น 2./m เพิ่มเป็น 2 เท่า เนื่องจาก ReLU Function ข้อมูลที่ติดลบ จะถูกทำให้เป็น 0 ทำข้อมูลหายไปครึ่งนึง 

In [44]:
w1 = torch.randn(m, nh) * math.sqrt(2./m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, nh)* math.sqrt(2./nh)
b2 = torch.zeros(nh)
w3 = torch.randn(nh, nh) * math.sqrt(2./nh)
b3 = torch.zeros(nh)
w4 = torch.randn(nh, 1) * math.sqrt(2./nh)
b4 = torch.zeros(1)

ลองดูค่า mean, std ของ Weight

In [45]:
w1.mean(), w1.std()

(tensor(0.0001), tensor(0.0508))

In [46]:
w2.mean(), w2.std()

(tensor(-0.0011), tensor(0.1402))

In [47]:
w3.mean(), w3.std()

(tensor(0.0019), tensor(0.1424))

In [48]:
w4.mean(), w4.std()

(tensor(-0.0125), tensor(0.1375))

Weight จะ mean, std เป็นอย่างไรเรายังไม่สนใจ

### ลองสมมติว่าเป็น Layer แรก

In [49]:
a1 = x_train @ w1
a1.mean(), a1.std()

(tensor(0.0492), tensor(1.3743))

ลองสมมติว่าเป็น Layer แรก + ReLU Activation

In [50]:
z1 = (x_train @ w1).clamp(0.)-0.5
z1.mean(), z1.std()

(tensor(0.0702), tensor(0.8312))

เราได้ Activation ของ Layer แรก ที่ใกล้เคียง mean = 0, std = 1 มากขึ้นอีก

### สร้างโมเดล

In [51]:
w1.g,b1.g,w2.g,b2.g,w3.g,b3.g,w4.g,b4.g = [None]*8
model = Model()
loss, yhat = model(x_train, y_train)
model.backward()

ตรวจเช็ค Mean และ Standard Deviation ของ Gradient ของ Weight ของแต่ละ Layer

In [52]:
w4.g.mean(), w4.g.std()

(tensor(0.6593), tensor(2.1650))

In [53]:
w3.g.mean(), w3.g.std()

(tensor(-0.0021), tensor(0.2243))

In [54]:
w2.g.mean(), w2.g.std()

(tensor(-0.0003), tensor(0.2305))

In [55]:
w1.g.mean(), w1.g.std()

(tensor(0.0011), tensor(0.2433))

เราได้ Gradient ที่ใกล้เคียง mean = 0, std = 1 มากขึ้นอีก

# 6. สรุป

1. วิธีการ Initialize ค่าของ Weight มีความสำคัญอย่างมาก ต่อ Gradient ซึ่งมีผลต่อการเทรน Neural Network
1. ยิ่ง Deep Neural Network ยิ่ง Deep มากขึ้นเท่าไร ยิ่งส่งผลต่อ Gradient เป็นทวีคูณ ถ้าน้อยก็จะน้อยไปเรื่อย ๆ จนเป็น 0 ถ้ามากก็จะมากจนเกินรับไหว เป็น Not a number (nan)
1. ในการเทรน Neural Network แล้ว Gradient เป็นส่วนสำคัญที่สุด ที่ช่วยให้เทรนสำเร็จ ได้โมเดลที่ดี จน Yann LeCun เสนอให้เรียกว่า Differentiable Programming แทน Deep Learning (Gradient คือ Diff ของหลายตัวแปร เช่น Matrix, Tensor)

# Credit

* https://course.fast.ai/videos/?lesson=8
* https://arxiv.org/abs/1502.01852
* http://proceedings.mlr.press/v9/glorot10a.html
* https://deeplearning.net